<a href="https://colab.research.google.com/github/ekqlsrla/ESAA-2/blob/main/HW/1031_%ED%8C%8C%EC%9D%B4%EC%8D%AC%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%ED%85%8D%EC%8A%A4%ED%8A%B8%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 파이썬 머신러닝 가이드
# 텍스트 분류 실습 - 20 뉴스그룹 분류


---
## 1. 텍스트 정규화

In [3]:
from sklearn.datasets import fetch_20newsgroups

news_data =fetch_20newsgroups(subset = 'all', random_state = 156)

In [ ]:
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [ ]:
#target 클래스 구성 확인

import pandas as pd
print('target 클래스의 값과 분포도 \n :', pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들 \n', news_data.target_names)

target 클래스의 값과 분포도 
 : 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [ ]:
#개별 데이터 확인

print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

* 뉴스그룹 기사의 내용뿐만 아니라 뉴스그룹 제목, 작성자, 소속, 이메일 등의 정보 가지고 있음
* **내용**을 제외하고 다른 정보 제거 -> `제목과 소속, 이메일 주소 등의 헤더와 푸터 정보들은 뉴스그룹 분류의 Target 클래스 값과 유사한 데이터를 가지고 있는 경우가 많음`
* `remove` 파라미터를 이용하면 제거할 수 있음

In [ ]:
from sklearn.datasets import fetch_20newsgroups

#subset='train'으로 학습용 데이터만 추출, remove = ('headers','footers','quotes')로 내용만 추출

train_news = fetch_20newsgroups(subset = 'train',remove = ('headers','footers','quotes'),
                   random_state = 156)
X_train = train_news.data
y_train = train_news.target

#subset='test'으로 학습용 데이터만 추출, remove = ('headers','footers','quotes')로 내용만 추출
test_news = fetch_20newsgroups(subset = 'test', remove = ('headers','footers','quotes'),
                               random_state = 156)

X_test = test_news.data
y_test = test_news.target

print('학습 데이터 크기 {0},테스트 데이터 크기 {1}'.format(len(train_news.data),len(test_news.data)))

학습 데이터 크기 11314,테스트 데이터 크기 7532


## 2. 피처 벡터화 변신과 머신러닝 모델 학습/예측/평가

* 테스트 데이터 피처 백터화 수행시 유의점
  * `CountVectorizer`를 적용할 때는 반드시 학습 데이터 이용해 `fit`이 수행된 `CountVectorizer` 객체를 이용해 테스트 데이털르 변환해야 한다는 것
  * 그래야만 학습 시 설정된 `CountVectorizer`의 피처 개수와 테스트 데이터를 `CountVectorizer`로 변환할 피처 개수가 같아짐

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

#Count Vectorization으로 피처 벡터화 변환 수행
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

#학습 데이터로 fit()된 CountVectorizer를 이용해 테스트 데이터를 피터 벡처화 변환 수행
X_test_cnt_vect = cnt_vect.transform(X_test)

print('학습 데이터 텍스트의 CountVectorizer Shape :', X_train_cnt_vect.shape)

학습 데이터 텍스트의 CountVectorizer Shape : (11314, 101631)


In [ ]:
#로지스틱 회귀를 적용해 분류 예측

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

#LogisticRegression을 이용해 학습/예측/평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect,y_train)
pred = lr_clf.predict(X_test_cnt_vect)
print('CountVectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

CountVectorized Logistic Regression의 예측 정확도는 0.608


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
#Count 기반에서 TF-IDF 기반으로 벡터화를 변경해 예측 모델 수행
from sklearn.feature_extraction.text import TfidfVectorizer

#TF-IDF 벡터화를 적용해 학습 데이터 세트와 테스트 데이터 세트 변환
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

#LogisticRegression을 이용해 학습/예측/평가 수행

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Logistic Regression의 예측 정확도는 {0:.3f}',format(accuracy_score(y_test,pred)))

TF-IDF Logistic Regression의 예측 정확도는 {0:.3f} 0.6736590546999469


* 텍스트 분석에서 성능을 향상시키는 중요한 2가지 방법
1. 최적의 ML 알고리즘 선택
2. 최상의 피처 전처리 수행

In [ ]:
#stop words 필터링을 추가하고 ngram을 기본 (1,1) -> (1,2)로 변경

tfidf_vect = TfidfVectorizer(stop_words = 'english',ngram_range= (1,2), max_df = 300)
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect,y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0:.3f}',format(accuracy_score(y_test,pred)))


TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0:.3f} 0.6922464152947424


In [ ]:
#GridSearchCV를 이용해 하이퍼 파라미터 최적화 수행

from sklearn.model_selection import GridSearchCV

params = {'C' : [0.01,0.1,1,5,10]}
grid_cv_lr = GridSearchCV(lr_clf, param_grid = params, cv = 3, scoring = 'accuracy',verbose = 1)
grid_cv_lr.fit(X_train_tfidf_vect,y_train)
print('Logistic Regression best C parameter : ', grid_cv_lr.best_params_)

pred = grid_cv_lr.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Logistic Regression best C parameter :  {'C': 10}
TF-IDF Vectorized Logistic Regression의 예측 정확도는 0.701


## 3. 사이킷런 파이프라인(Pipeline)사용 및 GridSearchCV와의 결합

: 피처 벡터화와 ML 알고리즘 학습/예측을 위한 코드 작성을 한 번에 진행할 수 있음

```
pipeline = Pipeline([('tfidf_vect',TfidfVectorizer(stop_words = 'english')),
                    ('lr_clf',LogisticRegression(random_state = 156))])
```

* `TfidfVectorizer` 객체를 **tfidf_score**라는 객체 변수명으로, `LogisticRegresion` 객체를 **lr_clf**라는 객체 변수 명으로 생성 한뒤  이 두개의 객체를 파이프라인으로 연결하는 Pipeline 객체 **pipeline**을 생성한다는 의미

In [2]:
from sklearn.pipeline import Pipeline
#TfidfVectorizer 객체를 tfidf_vect로, LogisticRegression 객체를 lr_clf로 생성하는 Pipeline 생성

pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words = 'english', ngram_range = (1,2), max_df = 300)),
    ('lr_clf',LogisticRegression(C=10))
])

NameError: ignored

In [ ]:
pipeline.fit(X_train,y_train)
pred = pipeline.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Pipeline을 통한 Logistic Regression의 예측 정확도는 0.701


* **TfidfVectorizer**의 파라미터와 **LogisticRegression**의 하이퍼 파라미터를 함께 최적화
  * `tfidf_vect_ngram_range`와 같이 하이퍼 파라미터명이 객체 변수명과 결합돼 제공

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words = 'english')),
    ('lr_clf', LogisticRegression())
])

#Pipeline에 기술된 각각의 객체 변수에 언더바 2개를 연달아 붙여 GridSearchCV에 사용될 이름과 값을 설정
params = {'tfidf_vect__ngram_range' : [(1,1),(1,2),(1,3)],
          'tfidf_vect__max_df': [100,300,700],
          'lr_clf__C' : [1,5,10]}

grid_cv_pipe = GridSearchCV(pipeline, param_grid = params, cv = 3, scoring = 'accuracy', verbose = 1)
grid_cv_pipe.fit(X_train,y_train)
print(grid_cv_pipe.best_params_, grid_cv_pipe.best_score_)

pred = grid_cv_pipe.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))